## 1.3 Merge

In this notebook, we will create one netcdf file out of the 175 nc files retrieved from CDS. 

### Import packages

In [1]:
##This is so variables get printed within jupyter
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [2]:
##import packages
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker


In [3]:
##We want the working directory to be the UNSEEN-open directory
pwd = os.getcwd() ##current working directory is UNSEEN-open/Notebooks/1.Download
pwd #print the present working directory
os.chdir(pwd+'/../../') # Change the working directory to UNSEEN-open
os.getcwd() #print the working directory

'/lustre/soge1/projects/ls/personal/timo/UNSEEN-open/Notebooks/1.Download'

'/lustre/soge1/projects/ls/personal/timo/UNSEEN-open'

## Concatenate SEAS5

The SEAS5 data consists of 175 netcdf files that we downloaded from CDS ([1.2 Retrieve](reference)). We concatenate those into one netcdf file here. We create a netcdf file containing the dimensions lat, lon, number (25 ensembles), time (35 years) and leadtime (5 initialization months). To generate this, we loop over lead times, and open all 35 years of the lead time and then concatenate those leadtimes. 

In [4]:
init_months = np.append(np.arange(9,13),1) ## Initialization months 9-12,1 (Sep-Jan)


In [8]:
SEAS5_ld1 = xr.open_mfdataset('../UK_example/SEAS5/' + '*01.nc',
                              combine='by_coords')  # Load the first lead time
SEAS5 = SEAS5_ld1  # Create the xarray dataset to concatenate over
for init_month in init_months[0:-1][::-1]:  ## Remove the first that we already have and invert so it is lead time 1-5
    print(init_month)
    SEAS5_ld = xr.open_mfdataset(
        '../UK_example/SEAS5/' + '*' + "%.2i" % init_month + '.nc',
        combine='by_coords')  # Load the first lead time to concatenate over
    SEAS5 = xr.concat([SEAS5, SEAS5_ld], dim='leadtime')

12
11
10
9


In [9]:
SEAS5 = SEAS5.assign_coords(leadtime = np.arange(5) + 2) # assign leadtime coordinates
SEAS5['tprate'] = SEAS5['tprate'] * 1000 * 3600 * 24 ## From m/s to mm/d
SEAS5['tprate'].attrs = {'long_name': 'rainfall',
 'units': 'mm/day',
 'standard_name': 'thickness_of_rainfall_amount'}
SEAS5
# SEAS5.load()
# SEAS5.to_netcdf('../UK_example/SEAS5/SEAS5.nc') # for future import

,Array,Chunk
Bytes,2.69 MB,15.40 kB
Shape,"(5, 35, 25, 11, 14)","(1, 1, 25, 11, 14)"
Count,1715 Tasks,175 Chunks
Type,float32,numpy.ndarray
